# LSTM Pipeline for Racing Line Prediction

In [1]:
# Quick check to ensure model will run on GPU
import torch
import cerberus 

2.6.0+cu126
CUDA available: True
Device: NVIDIA GeForce RTX 3070


In [2]:
# === Imports ===
import os
import numpy as np
import pandas as pd
from glob import glob
from tqdm.notebook import tqdm
import random
import torch as t
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.preprocessing import MinMaxScaler
import torch.serialization
torch.serialization.add_safe_globals([MinMaxScaler])
import matplotlib.pyplot as plt

In [3]:
# === Configuration ===

# config = {
#     "model_type": "lstm", # Options: "lstm", "cnn", "tcn", "transformer" 
#     "seed": 42,
#     "input_size": 6,
#     "output_size": 3,
#     "train_split": 0.8,
#     "num_epochs": 50,
#     "learning_rate": 0.001,
#     "batch_size": 64,
#     "hidden_size": 128,
#     "num_layers": 2,
#     "dropout": 0.2,
#     "seq_len": 150,
#     "patience": 10,
#     "pin_memory": True,
#     "bidirectional": False,
#     "device": "cuda" if torch.cuda.is_available() else "cpu",
#     "train_data_path": "./data/extracted_track_data/",
#     "test_data_path": "./data/testing_layouts/",
#     "model_save_path": "./models/testing_racing_line_lstm.pt",
#     "input_cols": ["left_x","left_y","left_z","right_x","right_y","right_z"],
#     "output_cols": ["x","y","z"]
# }

config = {
    "seed": 42,
    "input_size": 6,
    #model params
    "hidden1":128,
    "kern_size1":3,
    "kern_size2":3,
    "padding1": 2,
    "padding2": 4,
    "dilation1": 2,
    "dilation2": 4,
    "pos_head_sz": 64,
    "cont_head_sz": 64,
    #administrative stuff
    "device":"cuda",
    "input_cols": ["left_x","left_y","left_z","right_x","right_y","right_z"],
    "output_pos_cols": ["x","y","z"],
    "output_cont_cols": ["speed", "gas", "brake", "side_left", "side_right"],
    "patience": 10,
    #"train_data_path": "./data/extracted_track_data/",
    "train_data_path": "./data/testing_layouts/",
    "test_data_path": "./data/testing_layouts/",
    "model_save_path": "./models/testing_racing_proto_cerberus.pt",
    "pin_memory": True,
    #Training info,
    "num_epochs": 100,
    "long_epochs": 35,
    "medium_epochs": 35,
    "train_split": 0.8,
    "learning_rate": 0.001,
    "batch_size": 128,
    "long_seq_len": 600,
    "med_seq_len": 300,
    "short_seq_len": 150
}

random.seed(config["seed"])
np.random.seed(config["seed"])
torch.manual_seed(config["seed"])
torch.cuda.manual_seed(config["seed"])
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


In [ ]:
class RacingLineDataset(Dataset):
    def __init__(self, config):
        self.inputs, self.pos_targets, self.cont_targets = [], [], []
        self.scaler_x = MinMaxScaler()
        self.scaler_y = MinMaxScaler()
        self.scaler_z = MinMaxScaler()
        all_X, all_Y, all_Z = [], [], []
        train_files = sorted(glob(os.path.join(config["train_data_path"], "*.csv")))
        test_files = sorted(glob(os.path.join(config["test_data_path"], "*.csv")))

        # === First pass: Collect data for global fitting ===
        for file in train_files:
            df = pd.read_csv(file)
            X = df[config["input_cols"]].values
            Y = df[config["output_pos_cols"]].values
            Z = df[config["output_cont_cols"]].values
            all_X.append(X)
            all_Y.append(Y)
            all_Z.append(Z)
        for file in test_files:
            df = pd.read_csv(file)
            X = df[config["input_cols"]].values
            Y = df[config["output_pos_cols"]].values
            Z = df[config["output_cont_cols"]].values
            all_X.append(X)
            all_Y.append(Y)
            all_Z.append(Z)
        all_X = np.vstack(all_X)
        all_Y = np.vstack(all_Y)
        all_Z = np.vstack(all_Z)
        self.scaler_x.fit(all_X)
        self.scaler_y.fit(all_Y)
        self.scaler_z.fit(all_Z)

        # === Second pass: Normalize and extract sequences ===
        for file in train_files:
            df = pd.read_csv(file)
            is_circular = is_circular_track(df, config["output_cols"])
            X = self.scaler_x.transform(df[config["input_cols"]].values)
            Y = self.scaler_y.transform(df[config["output_pos_cols"]].values)
            Z = self.scaler_z.transform(df[config["output_cont_cols"]].values)

            for i in range(len(X)):
                self.inputs.append(get_centered_sequence(X, i, config["seq_len"], is_circular))
                self.pos_targets.append(Y[i])
                self.cont_targets.append(Z[i])
                
        self.inputs = torch.tensor(np.array(self.inputs), dtype=torch.float32).to(config["device"])
        self.pos_targets = torch.tensor(np.array(self.pos_targets), dtype=torch.float32).to(config["device"])
        self.cont_targets = torch.tensor(np.array(self.cont_targets), dtype=torch.float32).to(config["device"])
    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return self.inputs[idx].permute(1,0), self.pos_targets[idx], self.cont_targets[idx]
    

In [5]:
class Cerberus (torch.nn.Module):
    def __init__(self, config):
        super().__init__()

        #Thanks Resnet!
        class ResBlock(torch.nn.Module):
            def __init__(self, in_chans, out_chans, kern_size, padding, dilation):
                super().__init__()
                self.conv1 = torch.nn.Conv1d(in_chans, out_chans, kern_size, padding=padding, dilation=dilation)
                self.relu = torch.nn.ReLU()
                self.conv2 = torch.nn.Conv1d(out_chans, out_chans, kern_size, padding=padding, dilation=dilation)
    
                self.shortcut = torch.nn.Identity()
                if in_chans != out_chans:
                    self.shortcut = nn.Conv1d(in_chans, out_chans, kernel_size=1)
                
            def forward(self, x):
                residual = self.shortcut(x)
                return self.relu(self.conv2(self.relu(self.conv1(x))) + residual)
            
        
        
        self.encoder = torch.nn.Sequential(
            ResBlock(config["input_size"], config["hidden1"], config["kern_size1"], config["padding1"], config["dilation1"]),
            ResBlock(config["hidden1"], config["hidden1"], config["kern_size2"], config["padding2"], config["dilation2"])
        )

        #x, y, z
        self.position_head = t.nn.Sequential(
            torch.nn.Linear(config["hidden1"], config["pos_head_sz"]),
            torch.nn.ReLU(),
            torch.nn.Linear(config["pos_head_sz"], 3)
        )

        #speed, gas, brake, side-left, side-right
        self.control_head = t.nn.Sequential(
            torch.nn.Linear(config["hidden1"] + 3, config["cont_head_sz"]),
            torch.nn.ReLU(),
            torch.nn.Linear(config["cont_head_sz"], 5)
        )


            
        

    def forward(self, x):
        encoded = self.encoder(x)
        encoded = torch.mean(encoded, dim=2)

        position = self.position_head(encoded)
        control_in = torch.cat([encoded, position], dim=1)
        control = self.control_head(control_in)

        
        return position, control

In [6]:
# === Evaluation/Loss Functions ===
class PositionLoss(torch.nn.Module):
    def __init__(self, weight=1.0):
        super(PositionLoss, self).__init__()
        self.mse_loss = torch.nn.MSELoss()

    def forward(self, predicted, target):
        loss = self.mse_loss(predicted, target)
        return loss

#we can add penalties to encourage smoother and more realistic driving here
class ControlLoss(t.nn.Module):
    def __init__(self, weight=1.0):
        super(ControlLoss, self).__init__()
        self.mse_loss = torch.nn.MSELoss()
        
    def forward(self, predicted, target):
        loss = self.mse_loss(predicted, target)
        return loss

def evaluate_model(model, dataloader, pos_crit, con_crit, config):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for X_batch, Y_batch, Z_batch in dataloader:
            X_batch, Y_batch, Z_batch = X_batch.to(config["device"]), Y_batch.to(config["device"]), Z_batch.to(config["device"])
            pos, cont = model(X_batch)
            pos_loss = pos_crit(pos, Y_batch)
            con_loss = con_crit(cont, Z_batch)
            
            loss = pos_loss + 0.5 * con_loss
            total_loss += loss.item()
    return total_loss / len(dataloader)


In [7]:
# === Save and Load Model (with architecture) ===
def save_model(model, config):
    torch.save({
        "model_state_dict": model.state_dict(),
        "config": config,
        # "scaler_x": scaler_x,
        # "scaler_y": scaler_y,
        # "scaler_z": scaler_z,
    }, config["model_save_path"])

# def load_model(path):
#     checkpoint = torch.load(path, map_location=config["device"], weights_only=False)
#     cfg = checkpoint["config"]
#     model = RacingLineLSTMWithAttention(cfg["input_size"], cfg["hidden_size"],
#                                         cfg["output_size"], cfg["num_layers"],
#                                         cfg["dropout"], cfg["bidirectional"])
#     model.load_state_dict(checkpoint["model_state_dict"])
#     model.to(cfg["device"])
#     return model, checkpoint["scaler_x"], checkpoint["scaler_y"]

def load_model(path):
    checkpoint = torch.load(path, map_location=config["device"], weights_only=False)
    cfg = checkpoint["config"]
    #model = get_model(cfg)
    model = Cerberus(cfg)
    model.load_state_dict(checkpoint["model_state_dict"])
    model.to(cfg["device"])
    # scaler_x = checkpoint["scaler_x"]
    # scaler_y = checkpoint["scaler_y"]
    # scaler_z = checkpoint["scaler_z"]
    return model #, scaler_x, scaler_y, scaler_z


In [8]:
# === Training Function (with tqdm and validation) ===
def train_model(model, long_dataset, medium_dataset, short_dataset, config):
    optimizer = torch.optim.Adam(model.parameters(), lr=config["learning_rate"])
    # = hybrid_loss #nn.MSELoss()

    pos_crit = PositionLoss()
    con_crit = ControlLoss(weight=0.5)
    
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)
    best_val_loss = float("inf")
    train_losses = []
    val_losses = []
    early_stopping_patience = config["patience"]
    epochs_without_improvement = 0
    best_epoch = 0

    obar = tqdm(range(config["num_epochs"]), desc="Epochs")
    for epoch in obar:
        match epoch:
            case 0:
                train_loader = long_dataset[0]
                val_loader = long_dataset[1]
            case val if val == config["long_epochs"]:
                train_loader = medium_dataset[0]
                val_loader = medium_dataset[1]
            case val if val == config["medium_epochs"]:
                train_loader = medium_dataset[0]
                val_loader = medium_dataset[1]
            case _:
                pass
                
        model.train()
        running_loss = 0.0
        
        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{config['num_epochs']}", leave=False)
        for X_batch, Y_batch, Z_batch in pbar:
            X_batch, Y_batch, Z_batch = X_batch.to(config["device"]), Y_batch.to(config["device"]), Z_batch.to(config["device"])

            
            optimizer.zero_grad()
            p_pos, p_cont = model(X_batch)
            pos_loss, con_loss = pos_crit(p_pos, Y_batch), con_crit(p_cont, Z_batch)
            loss = pos_loss + 0.5 * con_loss
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            running_loss += loss.item()
            pbar.set_postfix({"Loss": running_loss / (pbar.n + 1)})

        train_losses.append(running_loss / len(train_loader))
        
        val_loss = evaluate_model(model, val_loader, pos_crit, con_crit, config)
        val_losses.append(val_loss)
        scheduler.step(val_loss)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            save_model(model, config)
            best_epoch = epoch+1
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1

        obar.set_postfix({"Train Loss": running_loss/len(train_loader), "Val Loss": val_loss, "Lr": scheduler.get_last_lr()[0], "best_epoch": best_epoch})

        if epochs_without_improvement >= early_stopping_patience:
            print(f"Early stopping triggered after {epoch+1} epochs.")
            break

    return train_losses, val_losses


In [ ]:
# === Full Pipeline ===
def run_pipeline(train_data_path=config["train_data_path"], test_data_path=config["test_data_path"]):
    print("Preparing dataset...")
    long_dataset = RacingLineDataset(
        train_data_path,
        test_data_path,
        config["long_seq_len"],
    )
    medium_dataset = RacingLineDataset(
        train_data_path,
        test_data_path,
        config["med_seq_len"],
    )
    short_dataset = RacingLineDataset(
        train_data_path,
        test_data_path,
        config["short_seq_len"],
    )

    datasets = [long_dataset, medium_dataset, short_dataset]
    scaler_x, scaler_y, scaler_z = long_dataset.scaler_x, long_dataset.scaler_y, long_dataset.scaler_z
    scalers = (scaler_x, scaler_y, scaler_z)
    # === 80/20 train/val split with reproducibility ===
    
    loaders = []
    
    for dataset in datasets:
        train_len = int(len(dataset) * config["train_split"])
        val_len = len(dataset) - train_len
        train_ds, val_ds = random_split(
            dataset,
            [train_len, val_len],
            generator=torch.Generator().manual_seed(config["seed"])
        )
        train_loader = DataLoader(train_ds, batch_size=config["batch_size"], shuffle=False, pin_memory=config["pin_memory"])
        val_loader = DataLoader(val_ds, batch_size=config["batch_size"], shuffle=False, pin_memory=config["pin_memory"])
        loaders.append([train_loader, val_loader])

    # train_loader = DataLoader(train_ds, batch_size=config["batch_size"], shuffle=False, pin_memory=config["pin_memory"])
    # val_loader = DataLoader(val_ds, batch_size=config["batch_size"], shuffle=False, pin_memory=config["pin_memory"])

    print(f"Initializing model: Cerberus")
    model = Cerberus(config=config).to(config["device"])

    print("Training started...")
    # train_losses, val_losses = train_model(model, train_loader, val_loader, config, scaler_x, scaler_y, scaler_z)
    train_losses, val_losses = train_model(model, loaders[0], loaders[1], loaders[2], config, scalers)
    print(f"Training complete. Model saved to {config['model_save_path']}")

    # === Plot learning curve ===
    plt.figure(figsize=(10, 5))
    plt.plot(train_losses, label="Train Loss")
    plt.plot(val_losses, label="Validation Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title("Learning Curve")
    plt.legend()
    plt.grid(True)
    plt.show()
    

In [10]:
# === Train Model ===
run_pipeline()

Preparing dataset...
Initializing model: Cerberus
Training started...


Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

Epoch 1/100:   0%|          | 0/15 [00:00<?, ?it/s]

NameError: name 'scaler_x' is not defined

In [ ]:
# === Inference on Testing Track Layouts (from coordinates only, doesnt take image for inference) ===
def print_feature_accuracy(preds, trues, scaler_y, feature_names):
    preds = np.array(preds)
    trues = np.array(trues)

    print(f"\nPer-Feature Accuracy (%):")
    print("-" * 60)
    for i, name in enumerate(feature_names):
        range_train = scaler_y.scale_[i]
        range_test = trues[:, i].max() - trues[:, i].min()

        if range_test == 0:
            print(f"{name:>16}: N/A (zero test range)")
            continue

        mean_error = np.mean(np.abs(preds[:, i] - trues[:, i]))
        acc_train = (1 - (mean_error / range_train)) * 100
        acc_test = (1 - (mean_error / range_test)) * 100

        acc_train = max(0.0, min(100.0, acc_train))
        acc_test = max(0.0, min(100.0, acc_test))

        print(f"{name:>16}: {acc_test:6.2f}% (layout-based)   {acc_train:6.2f}% (train-scale)")

# === Circular Tracks ===
def run_inference(data_folder, model_path):
    print("Loading model and scalers...")
    #model, scaler_x, scaler_y, _ = load_model(model_path)
    model = load_model(model_path)
    model.eval()

    print("Loading unseen layouts from:", data_folder)
    layout_files = sorted(glob(os.path.join(data_folder, "*.csv")))
    total_layouts = len(layout_files)
    print(f"Found {total_layouts} layout files.\n")

    for layout_index, layout_path in enumerate(layout_files):
        layout_name = os.path.basename(layout_path)
        print(f"[{layout_index + 1}/{total_layouts}] Predicting layout: {layout_name}")

        df = pd.read_csv(layout_path)
        X = df[config["input_cols"]].values
        Y = df[config["output_pos_cols"]].values
        #X_scaled = scaler_x.transform(X)
        n = len(X_scaled)
        preds_real = np.zeros_like(Y)
        trues_real = Y.copy()

        for i in tqdm(range(n), desc=f"[{layout_index + 1}/{total_layouts}]"):
            #seq = np.array([X_scaled[(i + j) % n] for j in range(config["seq_len"])])
            seq = np.array([X[(i + j) % n] for j in range(config["seq_len"])])
            X_tensor = torch.tensor(seq.reshape(1, config["seq_len"], -1), dtype=torch.float32).to(config["device"])
            X_tensor = X_tensor.permute(0, 2, 1)

            with torch.no_grad():
                pred, _ = model(X_tensor)
                pred_scaled = pred.cpu().squeeze().numpy()
                pred_real = scaler_y.inverse_transform(pred_scaled.reshape(1, -1))[0]
                print(pred_real)

            target_idx = (i + config["seq_len"]) % n
            preds_real[target_idx] += pred_real

        # === Plot X/Z comparison ===
        plt.figure(figsize=(12, 6))
        plt.plot(trues_real[:, 0], trues_real[:, 2], label="True", linewidth=2)
        plt.plot(preds_real[:, 0], preds_real[:, 2], label="Predicted", linewidth=2, linestyle="--")
        plt.title(f"X vs Z: {layout_name}")
        plt.xlabel("X Coordinate")
        plt.ylabel("Z Coordinate")
        plt.axis("equal")
        plt.grid(True)
        plt.legend()
        plt.show()

        # === Accuracy ===
        print_feature_accuracy(preds_real, trues_real, scaler_y, config["output_pos_cols"])

        # === Spatial Error (X/Z only)
        spatial_errors = np.linalg.norm(preds_real[:, [0, 2]] - trues_real[:, [0, 2]], axis=1)
        mean_spatial_error = np.mean(spatial_errors)
        max_spatial_error = np.max(spatial_errors)
        print(f"Mean X/Z spatial error: {mean_spatial_error:.2f}m, Max: {max_spatial_error:.2f}m\n")

# === Non-Circular Tracks ===
# def run_inference(data_folder, model_path):
#     print("Loading model and scalers...")
#     model, scaler_x, scaler_y = load_model(model_path)
#     model.eval()

#     print("Loading unseen layouts from:", data_folder)
#     layout_files = sorted(glob(os.path.join(data_folder, "*.csv")))
#     total_layouts = len(layout_files)
#     print(f"Found {total_layouts} layout files.\n")

#     for layout_index, layout_path in enumerate(layout_files):
#         layout_name = os.path.basename(layout_path)
#         print(f"[{layout_index + 1}/{total_layouts}] Predicting layout: {layout_name}")

#         # === Load layout data ===
#         df = pd.read_csv(layout_path)
#         X = df[config["input_cols"]].values
#         Y = df[config["output_cols"]].values
#         X_scaled = scaler_x.transform(X)
#         n = len(X_scaled)
#         seq_len = config["seq_len"]

#         preds_real = np.zeros_like(Y)
#         trues_real = Y.copy()

#         for i in tqdm(range(n), desc=f"[{layout_index + 1}/{total_layouts}]"):
#             # Pad first `seq_len` frames with the first frame
#             if i < seq_len:
#                 pad = np.repeat(X_scaled[0:1], seq_len - i, axis=0)
#                 seq = np.vstack([pad, X_scaled[0:i]])
#             else:
#                 seq = X_scaled[i - seq_len:i]

#             X_tensor = torch.tensor(seq.reshape(1, seq_len, -1), dtype=torch.float32).to(config["device"])

#             with torch.no_grad():
#                 pred_scaled = model(X_tensor).cpu().squeeze().numpy()
#                 pred_real = scaler_y.inverse_transform(pred_scaled.reshape(1, -1))[0]

#             preds_real[i] = pred_real

#         # === Plotting ===
#         plt.figure(figsize=(12, 6))
#         plt.plot(trues_real[:, 0], trues_real[:, 2], label="True", linewidth=2)
#         plt.plot(preds_real[:, 0], preds_real[:, 2], label="Predicted", linewidth=2, linestyle="--")
#         plt.title(f"X vs Z Trajectory: {layout_name}")
#         plt.xlabel("X Coordinate")
#         plt.ylabel("Z Coordinate")
#         plt.axis("equal")
#         plt.grid(True)
#         plt.legend()
#         plt.show()

#         # === Accuracy report ===
#         print_feature_accuracy(preds_real, trues_real, scaler_y, config["output_cols"])

#         # === Spatial error metrics (just X/Z)
#         spatial_errors = np.linalg.norm(preds_real[:, [0, 2]] - trues_real[:, [0, 2]], axis=1)
#         mean_spatial_error = np.mean(spatial_errors)
#         max_spatial_error = np.max(spatial_errors)
#         print(f"Mean X/Z spatial error: {mean_spatial_error:.2f}m, Max: {max_spatial_error:.2f}m\n")

# === Run it ===
run_inference(data_folder="./data/testing_layouts", model_path="./models/testing_racing_proto_cerberus.pt")
